## Etiquetado de tweets

In [59]:
import pandas as pd
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [60]:
system_prompt = """You are a data scientist labelling tweets. Given a tweet, classify it into one of the following categories:

Tesla Products
Customer Experience
Performance & Innovation
Financial News
Environmental Impact
Industry News
Charging Infrastructure

If the tweet does not clearly fit into any of these categories or is not relevant, classify it as "Not relevant".

Example:

Tweet:

Tesla's new Model S Plaid is the fastest production car ever made!

Category:

Tesla Products

Example:

Tweet:

I'm having a terrible time with Tesla customer service. They've been ignoring my emails for weeks.

Category:

Customer Experience

I want you to be very rigorous with the labelling. You have to respond with the following phrase: Category: [response]"""

In [61]:
df = pd.read_csv('../../data/clean/tesla_tweets_unlabelled.csv')
test_df = df.sample(20)
df.head()

,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,Crypto news Bitcoin Whales flying motorbikes w...
1,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
2,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
3,https://twitter.com/ElTendies/status/151317393...,2022-04-10 19:45:00,Tesla A Trillion Dollar Company Worlds Largest...
4,https://twitter.com/LauraCory2013/status/15131...,2022-04-10 19:45:00,chargingstations area I dont Tesla Powerwall a...


In [62]:
df_labelled = pd.DataFrame(columns=['Link', 'Date', 'Text', 'Category'])
progress = 0

for index, row in test_df.iterrows():
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row['Text']}
      ],
        max_tokens=10,
        temperature=0.9,
    )
    answer = completion.choices[0].message['content'].replace('Category: ', '')
    df_labelled.loc[len(df_labelled.index)] = [row['Link'], row['Date'], row['Text'], answer]
    progress += 1
    if progress % 10 == 0:
      print(f"Progress: {progress}/{len(test_df.index)}")



Progress: 10/20


KeyboardInterrupt: 

In [ ]:
df_aux = pd.read_csv('../../data/clean/tesla_tweets_labelled.csv')

#comprobar que ninguna fila coincide con la que ya esta en el df_aux

mask = df_labelled['Link'].isin(df_aux['Link'])
df_labelled = df_labelled[~mask]

pd.concat([df_aux, df_labelled]).to_csv('../../data/clean/tesla_tweets_labelled.csv', index=False)

In [ ]:
df = pd.read_csv('../../data/clean/tesla_tweets_labelled.csv')
print(df.shape)
df['Category'].value_counts()

(995, 4)


Category
Not relevant                420
Financial News              177
Tesla Products              161
Industry News               118
Performance & Innovation     38
Environmental Impact         38
Charging Infrastructure      31
Customer Experience          10
Not relevant.                 1
Not Relevant                  1
Name: count, dtype: int64